In [2]:
import pandas as pd
import duckdb
import openai
import time 
import os

/Users/weizhuoqun/Documents/ds projects/Lang2SQL/openai_api/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Load in data

We use the [Chicago crime dataset](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/about_data) from 2021, 2022, and 2023

In [3]:
path = "./data"
files = [x for x in os.listdir(path = path)]
print(files)

['Crimes_-_2021_20240417.csv', 'Crimes_-_2022_20240417.csv', 'Crimes_-_2023_20240417.csv']


In [4]:
# read in and concat data 
chicago_crime = pd.concat((pd.read_csv(path +"/" + f) for f in files), ignore_index=True)

# chicago_crime.shape #(709386, 22)
chicago_crime.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,25953,JE240540,05/24/2021 03:06:00 PM,020XX N LARAMIE AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,...,36.0,19,01A,1141387.0,1913179.0,2021,11/18/2023 03:39:49 PM,41.917838,-87.755969,"(41.917838056, -87.755968972)"
1,26038,JE279849,06/26/2021 09:24:00 AM,062XX N MC CORMICK RD,0110,HOMICIDE,FIRST DEGREE MURDER,PARKING LOT,True,False,...,50.0,13,01A,1152781.0,1941458.0,2021,11/18/2023 03:39:49 PM,41.995219,-87.713355,"(41.995219444, -87.713354912)"
2,12342615,JE202211,04/17/2021 03:20:00 PM,081XX S PRAIRIE AVE,0325,ROBBERY,VEHICULAR HIJACKING,RESIDENCE,True,False,...,6.0,44,03,1179448.0,1851073.0,2021,09/14/2023 03:41:59 PM,41.746626,-87.618032,"(41.746626309, -87.618031954)"
3,26262,JE366265,09/08/2021 04:45:00 PM,047XX W HARRISON ST,0110,HOMICIDE,FIRST DEGREE MURDER,CAR WASH,True,False,...,24.0,25,01A,1144907.0,1896933.0,2021,09/14/2023 03:41:59 PM,41.873191,-87.743447,"(41.873191445, -87.743446563)"
4,13209581,JG422927,08/01/2021 12:00:00 AM,012XX E 78TH ST,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,APARTMENT,False,False,...,8.0,45,17,NaN,NaN,2021,09/14/2023 03:43:09 PM,NaN,NaN,NaN


# Prompt Engineering

The OpenAI API documentation provides a [recommendation](https://platform.openai.com/examples/default-sql-translate) for how to set the system and user components in a prompt when requesting to generate an SQL code:  
For context, 
* `system` is the context, e.g. "Given the following SQL tables, your job is to write queries given a user’s request."  
```
CREATE TABLE Orders (
  OrderID int,
  CustomerID int,
  OrderDate datetime,
  OrderTime varchar(8),
  PRIMARY KEY (OrderID)
);
...
```
* `user` is the request, e.g. "Write a SQL query which computes the average total order value for all orders on 2023-04-01."

### Step 1: Create a function `prompt_generator(table_name, query)` that generates the system and user prompts like the example above.
* Parameters:
1. table name
2. user query request
* Output:
completed system and user prompt messages

Use the `DuckDB` library to handle the pandas' df as it was a SQL table and extract its column names and types. 

In [5]:
# Extract info
duckdb.sql("DESCRIBE SELECT * FROM chicago_crime")

┌──────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │ column_type │  null   │   key   │ default │  extra  │
│       varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ID                   │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Case Number          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Date                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Block                │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ IUCR                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Primary Type         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Description          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Location Description │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Arrest               │ BOOLEAN     │ YES     │ NUL

In [113]:
def prompt_generator(table_name, user_query):

    class message:
        def __init__(message, context, user, column_name):
            message.context = context
            message.user = user
            message.column_names = column_name_list
    
    # Parse the table to extract the first two columns & create a string
    table_des = duckdb.sql("DESCRIBE SELECT * FROM chicago_crime")
    column_name_list = []
    column_name_type_list = []
    for i in range(len(table_des.df().column_name)):
        column_name = table_des.df().column_name[i]
        column_type = table_des.df().column_type[i]
        column_name_type = column_name + ' ' + column_type
        column_name_type_list.append(column_name_type)
        column_name_list.append(column_name)

    column_name_type_string = str(column_name_type_list).replace('[','(').replace(']',')').replace("\'", "")

    # create context and user string
    context = """
    Given the following SQL tables, your job is to write queries given a user’s request.
    CREATE TABLE {} {} \n
    """.format(table_name, column_name_type_string) 
    user = "Write a SQL query that returns {}".format(user_query)
    
    prompt = message(context = context, user = user, column_name = column_name)

    return prompt

In [16]:
table_name = "chicago_crime"
user_query = "the number of cases in each year, order by year in the ascending order."

print(prompt_generator(table_name, user_query).context)
print(prompt_generator(table_name, user_query).user)


    Given the following SQL tables, your job is to write queries given a user’s request.
    CREATE TABLE chicago_crime (ID BIGINT, Case Number VARCHAR, Date VARCHAR, Block VARCHAR, IUCR VARCHAR, Primary Type VARCHAR, Description VARCHAR, Location Description VARCHAR, Arrest BOOLEAN, Domestic BOOLEAN, Beat BIGINT, District BIGINT, Ward DOUBLE, Community Area BIGINT, FBI Code VARCHAR, X Coordinate DOUBLE, Y Coordinate DOUBLE, Year BIGINT, Updated On VARCHAR, Latitude DOUBLE, Longitude DOUBLE, Location VARCHAR) 

    
Write a SQL query that returns the number of cases in each year, order by year in the ascending order.


The output of the prompt_generator function was designed to fit the OpenAI API ChatCompletion.create function arguments, which we will review in the next section.

## Step2: Interact with the OpenAI API

The [openai](https://github.com/openai/openai-python) library enables access to the OpenAI REST API. We will use the library to connect to the API and send GET requests with our prompt.

a) Connecting to the API

In [25]:
openai.api_key = "sk-proj-iTkXmrdYl4TAxAonBFaBT3BlbkFJ0qqU4tDjKNTSIkWtsiVS"

In [26]:
# all available LLMs
openai_api_models = pd.DataFrame(openai.Model.list()["data"])

openai_api_models.head

<bound method NDFrame.head of                              id object     created         owned_by
0                     whisper-1  model  1677532384  openai-internal
1                   davinci-002  model  1692634301           system
2                      dall-e-2  model  1698798177           system
3                 gpt-3.5-turbo  model  1677610602           openai
4                 tts-1-hd-1106  model  1699053533           system
5                      tts-1-hd  model  1699046015           system
6            gpt-3.5-turbo-0125  model  1706048358           system
7             gpt-3.5-turbo-16k  model  1683758102  openai-internal
8        gpt-3.5-turbo-16k-0613  model  1685474247           openai
9                         tts-1  model  1681940951  openai-internal
10                     dall-e-3  model  1698785189           system
11           gpt-3.5-turbo-0613  model  1686587434           openai
12           gpt-3.5-turbo-0301  model  1677649963           openai
13  gpt-3.5-turbo-

For text generation, we use `gpt-3.5-turbo` model which is a series of models that keep getting updated. By default, if the model version is not specified, the API will point out to the most recent stable release. GPT-3.5 Turbo models can understand and generate natural language or code and have been optimized for chat using the Chat Completions API but work well for non-chat tasks as well.

To send a `GET` request with our prompt, we will use the `ChatCompletion.create` function. The function has many arguments, and we will use the following ones:

- `model` - The model ID to use, a full list available [here](https://platform.openai.com/docs/models/gpt-3-5-turbo)
- `messages` - A list of messages containing the prompt
- `temperature` - Manage the randomness or determinism of the process output by setting the sampling temperature level. The temperature level accepts values between 0 and 2. When the argument value is higher, the output becomes more random. Conversely, when the argument value is closer to 0, the output becomes more deterministic (reproducible)
- `max_tokens` - The maximum number of tokens to generate in the completion
  
The full list of the function arguments available on the API documentaiton.

b) Create prompt using the `prompt_generator` function 

In [114]:
query = "the average number of cases in each year and the year-over-year percentage of the number of cases, order by year in the ascending order."
prompt = prompt_generator(table_name = "chicago_crime", user_query = query)
print(prompt.context)
print(prompt.user)



    Given the following SQL tables, your job is to write queries given a user’s request.
    CREATE TABLE chicago_crime (ID BIGINT, Case Number VARCHAR, Date VARCHAR, Block VARCHAR, IUCR VARCHAR, Primary Type VARCHAR, Description VARCHAR, Location Description VARCHAR, Arrest BOOLEAN, Domestic BOOLEAN, Beat BIGINT, District BIGINT, Ward DOUBLE, Community Area BIGINT, FBI Code VARCHAR, X Coordinate DOUBLE, Y Coordinate DOUBLE, Year BIGINT, Updated On VARCHAR, Latitude DOUBLE, Longitude DOUBLE, Location VARCHAR) 

    
Write a SQL query that returns the average number of cases in each year and the year-over-year percentage of the number of cases, order by year in the ascending order.


c) Transform the above prompt into the structure of the `ChatCompletion.create` function messages argument:

In [71]:
message = [
    {
      "role": "system",
      "content": prompt.context
    },
    {
      "role": "user",
      "content": prompt.user
    }
    ]

d) Send the prompt (i.e., the `message` object) to the API using the `ChatCompletion.create` function

In [96]:
response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = message,
    temperature = 0, 
    max_tokens = 256
)

We will set the temperature argument to 0 to ensure high reproducibility and limit the number of tokens in the text completion to 256. The function returns a JSON object with the text completion, metadata, and other information:

In [73]:
print(response)

{
  "id": "chatcmpl-9F9XGBgpFXw5QDT5vfwcq7mTDOT7h",
  "object": "chat.completion",
  "created": 1713398602,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "```sql\nSELECT \n    Year,\n    AVG(CASES) AS Avg_Cases,\n    ROUND((AVG(CASES) - LAG(AVG(CASES), 1) OVER (ORDER BY Year)) / LAG(AVG(CASES), 1) OVER (ORDER BY Year) * 100, 2) AS YoY_Percentage\nFROM\n    (SELECT \n        Year,\n        COUNT(*) AS CASES\n    FROM \n        chicago_crime\n    GROUP BY \n        Year) subquery\nGROUP BY \n    Year\nORDER BY \n    Year ASC;\n```"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 155,
    "completion_tokens": 125,
    "total_tokens": 280
  },
  "system_fingerprint": "fp_c2295e73ad"
}


e) Extract SQL query from the response variable

In [97]:
sql = response["choices"][0]["message"]["content"]

print(sql)

```sql
SELECT 
    Year,
    AVG(CASES) AS Avg_Cases,
    ROUND((AVG(CASES) / LAG(AVG(CASES), 1) OVER (ORDER BY Year) - 1) * 100, 2) AS YoY_Percentage
FROM
    (SELECT 
        Year,
        COUNT(DISTINCT Case_Number) AS CASES
    FROM 
        chicago_crime
    GROUP BY 
        Year) subquery
GROUP BY 
    Year
ORDER BY 
    Year ASC;
```


Looks correct!! 

f) Using the duckdb.sql function to run the SQL code:

In [98]:
duckdb.sql(sql[6:-3]).show()

BinderException: Binder Error: Referenced column "Case_Number" not found in FROM clause!
Candidate bindings: "chicago_crime.Case Number"

# Put all the code together

One thing to note about the returned SQL code from the `ChatCompletion.create` function is that the variable does not return with quotes. That might be an issue when the variable name in the query combines two or more words. For example, using a variable such as `Case Number` or `Primary Type` from the `chicago_crime` inside a query without using quotes will result in an error.

a) Introduce a helper function to wrap all column names in the SQL query returned from `ChatCompletion.create` function in quotes

In [181]:
import re

def add_quotes(query, col_names):
    for i in col_names:    
        indices = [m.start() for m in re.finditer(i, query)]
        count = 0
        for y in indices:
            x = y + count
            if x == 0 or (query[x-1] != "'" and query[x-1] != "\"" and (query[x+len(i)] == " " or query[x+len(i)] == "," or query[x+len(i)] == "\n") and query[x-1] == " "):
                count += 2
                query = query[:x] + "\"" + i + "\"" + query[x + len(i):]
        # if i in query:
        #     l = query.find(i)
        #     print(l)
        #     if query[l + len(i)] != " " and query[l + len(i)] == "
        #     if query[l-1] != "'" and query[l-1] != '"' and query[l-1] == " " and query[l+len(i)] == " ": 
        #         query = str(query).replace(i, "'" + i + "'") 

    return(query)

In [174]:
sql = '\nSELECT COUNT(*) AS Arrested_Cases, Case Number\nFROM chicago_crime\nWHERE Case Number = 1234;\n'

In [182]:
add_quotes(sql, col_names=prompt.column_names)

'\nSELECT COUNT(*) AS Arrested_Cases, "Case Number"\nFROM chicago_crime\nWHERE "Case Number" = 1234;\n'

b) Build the Lang2SQL funtion that stitch all three functions that we have built so far, i.e. `prompt_generation`, `ChatCompletion.create`, and `add_quote` together.  

The parameters are: `api_key`, `table_name`, `query`, and some model-related parameters  
Returns: returns an object with the prompt, the API response, and the parsed query

In [107]:
def prompt_generator(table_name, user_query):

    class message:
        def __init__(message, context, user, column_name):
            message.context = context
            message.user = user
            message.column_names = column_name_list
    
    # Parse the table to extract the first two columns & create a string
    table_des = duckdb.sql("DESCRIBE SELECT * FROM chicago_crime")
    column_name_list = []
    column_name_type_list = []
    for i in range(len(table_des.df().column_name)):
        column_name = table_des.df().column_name[i]
        column_type = table_des.df().column_type[i]
        column_name_type = column_name + ' ' + column_type
        column_name_type_list.append(column_name_type)
        column_name_list.append(column_name)

    column_name_type_string = str(column_name_type_list).replace('[','(').replace(']',')').replace("\'", "")

    # create context and user string
    context = """
    Given the following SQL tables, your job is to write queries given a user’s request.
    CREATE TABLE {} {} \n
    """.format(table_name, column_name_type_string) 
    user = "Write a SQL query that returns {}".format(user_query)
    
    prompt = message(context = context, user = user, column_name = column_name)

    return prompt

In [121]:
def Lang2SQL(api_key, 
             table_name, 
             query, 
             model = "gpt-3.5-turbo", 
             temperature = 0, 
             max_tokens = 256, 
             # frequency_penalty = 0,
             # presence_penalty= 0
            ):
    class response:
        def __init__(output, message, response, sql):
            output.message = message
            output.response = response
            output.sql = sql

    # API key
    openai.api_key = api_key

    # prompt_generator: generate prompt based on the table name and the user question
    m = prompt_generator(table_name = table_name, user_query = query)

    # preparing message to be fed into ChatCompletion.create function
    message = [
    {
      "role": "system",
      "content": m.context
    },
    {
      "role": "user",
      "content": m.user
    }
    ]

    # sent request to OpenAI API
    openai_response = openai.ChatCompletion.create(
        model = model,
        messages = message,
        temperature = temperature,
        max_tokens = max_tokens,
        # frequency_penalty = frequency_penalty,
        # presence_penalty = presence_penalty
    )

    # parse response to get SQL query
    sql_query = add_quotes(query = openai_response["choices"][0]["message"]["content"][6:-3], col_names = m.column_names)

    output = response(message = m, response = openai_response, sql = sql_query)


    return output
    

Now let's test it out! 

In [183]:
query = "How many cases ended up with arrest?"
response = Lang2SQL(api_key = openai.api_key, table_name = "chicago_crime", query = query)

In [184]:
response.sql

'\nSELECT COUNT(*) AS Arrested_Cases\nFROM chicago_crime\nWHERE "Arrest" = TRUE;\n'

In [185]:
duckdb.sql(response.sql).show()

┌────────────────┐
│ Arrested_Cases │
│     int64      │
├────────────────┤
│          85530 │
└────────────────┘

